In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"prakash472","key":"e09c0084da5ef1992a912c0760649015"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
#change permission
!chmod 600 /root/.kaggle/kaggle.json
#Load the dataset from api
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

 66% 17.0M/25.7M [00:00<00:00, 77.3MB/s]
100% 25.7M/25.7M [00:00<00:00, 74.1MB/s]


In [ ]:
!unzip /content/imdb-dataset-of-50k-movie-reviews.zip

Archive:  /content/imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 51kB 7.6MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.7-cp36-none-any.whl size=30539 sha256=f21f4afbd87bcdd4b4f407c859027db34298864bda02695524ad7d4c2e51a42a
  Stored in directory: /root/.cache/pip/wheels/e1/f8/e2/b98f79a6b8cc898d8e4102b83acb8a098df7d27500a2bac912
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7304 sha256=79f4c29de0cf957083ff3e2ae169c9f329c7cd3900fbbc0d77ff48f0cdf0803f
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=4c10c69601cbd59d9153088cee83018299a7bcecfd368f11ac8fdd19be011bcf
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow
     |████████████████████████████████| 1.1MB 24.5MB/s 


In [ ]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_hub as hub
import bert

In [ ]:
#reading dataset
import pandas as pd
movie_reviews=pd.read_csv("/content/imdb-dataset-of-50k-movie-reviews.zip")
movie_reviews.isnull().values.any()
movie_reviews.shape

(50000, 2)

In [ ]:
movie_reviews.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
import re
TAG_RE = re.compile(r'<[^>]+>')

def remove_tags(text):
    return TAG_RE.sub('', text)  

In [ ]:
def preprocess_text(sen):
  
  #Removing HTML tags
  sentence=remove_tags(sen)

  #Select only alphabet characters
  sentence=re.sub('[^A-Za-z]',' ',sentence)

  #Removing multiple spaces
  sentence=re.sub(r"\s+"," ", sentence)

  #Remove the single character
  #sentence=re.sub(r"\s+[A-Za-z]+\s+",' ', sentence)

  return sentence




In [ ]:
print(preprocess_text("Ram123 is a boy@123e"))

Ram is a boy e


674

In [ ]:
# The above text revies consists of various tags and number so lets clean it
reviews=[]
sentences=list(movie_reviews["review"])
for sentence in sentences:
  reviews.append(preprocess_text(sentence))

In [ ]:
print(movie_reviews.columns.values)

['review' 'sentiment']


In [ ]:
# Here we have sentiment in form of categorical data so we convert it into numbers. Since, only
# one can define both positive and negative.
sentiment=pd.get_dummies(movie_reviews['sentiment'],drop_first=True)
sentiment.head()


,positive
0,1
1,1
2,1
3,0
4,1


In [ ]:
# We do not require a dataframe but rather a list so
import numpy as np
"""
if type(sentiment) != list:
  sentiment=list(sentiment["positive"])
  print(len(sentiment))
  print(sentiment[3])

sentiments=[]
sentiments_sentences=list(movie_reviews["sentiment"])
for sentiment_sentence in sentiments_sentences:
  sentiments.append(sentiment_sentence)
sentiment=np.array(sentiments)
sentiment.shape
"""

(50000,)

In [ ]:
# Here reviews variable contains the reviews and sentiment variable contains the sentiment. 
# Let's check for 4th review or 3rd index review. From above it is negative one.
print(reviews[3]) 

Basically there s a family where a little boy Jake thinks there s a zombie in his closet his parents are fighting all the time This movie is slower than a soap opera and suddenly Jake decides to become Rambo and kill the zombie OK first of all when you re going to make a film you must Decide if its a thriller or a drama As a drama the movie is watchable Parents are divorcing arguing like in real life And then we have Jake with his closet which totally ruins all the film I expected to see a BOOGEYMAN similar movie and instead i watched a drama with some meaningless thriller spots out of just for the well playing parents descent dialogs As for the shots with Jake just ignore them 


In [ ]:
"""
Here Bert creates its own tokenizer. Here, we have our data. Now let's create the BERT representation
of our text. In order to use BERT word embeddings we need to tokenize our text. In BERT we have
[cls]--classification 
TEXT
[SEP]
tokens. 
Since BERT has its own tokenizer we create the bert tokenizer. It can be done as
"""
''' 
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)
 '''

In [ ]:
''' tokenizer.tokenize("Hey my Name is Prakash. Don't be so judgemental") '''

In [ ]:
#Here, the tokens have been created. So we can also get their ids of tokens as
''' tokenizer.convert_tokens_to_ids(tokenizer.tokenize("Hey my Name is Prakash. Don't be so judgemental"))
 '''

In [ ]:
# Now let's tokenize every sentence from our reviews
def tokenize_reviews(review):
  return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(review))

tokenized_reviews=[tokenize_reviews(review) for review in reviews]

In [ ]:
"""
In the bert model we should have equal input length or the tokens should be of same size. In order
to do that we can add the pre padding zeros or post adding zeros. Since our sentence length is 
too large performing them on a single instance will create a large matrix. Since, we will be 
training in batches we can adjust the tokens according to batches. Now, Let's create our final
data which consists of review, length of review and sentiment of that review. We have list of 
list.[[review_1,length_1,sentiment_1],
      [review_2,length_2,sentiment_2],
]
"""
reviews_with_len=[[review, len(review),sentiment[i]] for i,review in enumerate(tokenized_reviews)]

In [ ]:
''' import random
#Just shuffling the data
random.shuffle(reviews_with_len) '''

In [ ]:
# Here, we have inputs but lets sort them according to their length so that padding of 
# tokens becomes easy when we work with batches. So, it is useful from preventing creation of sparse matrix and fast processing.
# The lambda function sorts with entry according to length of the reviews
#reviews_with_len.sort(key=lambda x:x[1])

In [ ]:
# Once the data are sorted according to the length of the reviews there is no further need to store the length. We can remove it
#sorted_reviews=[(review[0],review[2]) for review in reviews_with_len]

In [ ]:
# Lets convert it into tf format
#processed_dataset=tf.data.Dataset.from_generator(lambda: sorted_reviews, output_types=(tf.int32, tf.int32))

In [ ]:
#type(processed_dataset)

tensorflow.python.data.ops.dataset_ops.FlatMapDataset

In [ ]:
''' BATCH_SIZE=32
batched_dataset=processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ),()))
next(iter(batched_dataset)) '''

(<tf.Tensor: shape=(32, 22), dtype=int32, numpy=
 array([[ 3078,  5436,  3078,  3257,  3532,  7613,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [ 3191,  1996,  2338,  5293,  1996,  3185,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [ 2062, 23873,  3993,  2062, 11259,  2172,  2172,  2062, 14888,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [ 2054,  1037,  5896,  2054,  1037,  2466,  2054,  1037,  6752,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [ 2023,  3185,  2003,  6659,  2021,  2009,  2038,  2070,  2204,
          3896,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0],
        [ 8235,  1998,  3048,  4616,  201

In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip

--2020-11-18 02:14:52--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.7.240, 172.217.13.80, 172.253.122.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.7.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   164MB/s    in 2.4s    

2020-11-18 02:14:54 (164 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]



In [ ]:
!unzip uncased_L-12_H-768_A-12.zip

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
import os
os.makedirs("model",exist_ok=True)

In [ ]:
!mv uncased_L-12_H-768_A-12/ model

In [ ]:
bert_model_name="uncased_L-12_H-768_A-12"
bert_ckpt_dir=os.path.join("/content/model",bert_model_name)
bert_ckpt_file=os.path.join(bert_ckpt_dir,"bert_model.ckpt")
bert_config_file=os.path.join(bert_ckpt_dir,"bert_config.json")

In [ ]:
bert_ckpt_file

'/content/model/uncased_L-12_H-768_A-12/bert_model.ckpt'

In [ ]:
tokenizer=bert.bert_tokenization.FullTokenizer(vocab_file=os.path.join(bert_ckpt_dir,"vocab.txt"),do_lower_case=True)

In [ ]:
tokens=tokenizer.tokenize("I am leaving here. Can't go out from here!")
print(len(tokens))
tokenizer.convert_tokens_to_ids(tokens)

13


[1045, 2572, 2975, 2182, 1012, 2064, 1005, 1056, 2175, 2041, 2013, 2182, 999]

In [ ]:
# Now let's tokenize every sentence from our reviews
MAX_LENGTH=256
def tokenize_reviews(reviews):
  tokenized_reviews=[]
  for review in reviews:
    tokens=tokenizer.tokenize(review)
    if len(tokens)>MAX_LENGTH-2:
      tokens=tokens[:MAX_LENGTH-2]
    tokens = ["[CLS]"] + tokens + ["[SEP]"]
    token_ids=tokenizer.convert_tokens_to_ids(tokens)
    tokenized_reviews.append(token_ids)
  return tokenized_reviews
tokenized_reviews=tokenize_reviews(reviews)

In [ ]:
print(tokenized_reviews[2])

[101, 1045, 2245, 2023, 2001, 1037, 6919, 2126, 2000, 5247, 2051, 2006, 1037, 2205, 2980, 2621, 5353, 3564, 1999, 1996, 2250, 22442, 4258, 1998, 3666, 1037, 2422, 18627, 4038, 1996, 5436, 2003, 21934, 24759, 6553, 2021, 1996, 7982, 2003, 25591, 1998, 1996, 3494, 2024, 5622, 2912, 3468, 2130, 1996, 2092, 7852, 6878, 7642, 6359, 2096, 2070, 2089, 2022, 9364, 2043, 2027, 5382, 2023, 2003, 2025, 2674, 2391, 3891, 13449, 1045, 2245, 2009, 2001, 6947, 2008, 13703, 5297, 2003, 2145, 3929, 1999, 2491, 1997, 1996, 2806, 2116, 1997, 2149, 2031, 4961, 2000, 2293, 2023, 2001, 1996, 2087, 1045, 1040, 4191, 2012, 2028, 1997, 13703, 1055, 22092, 1999, 2086, 8108, 1045, 2360, 1037, 5476, 2096, 1045, 2310, 2196, 2042, 7622, 2007, 11862, 13093, 3385, 1999, 2023, 2016, 3266, 2000, 4309, 2091, 2014, 7916, 3746, 1998, 5598, 2157, 2046, 1037, 2779, 2021, 24462, 2402, 2450, 2023, 2089, 2025, 2022, 1996, 4410, 13713, 1997, 2010, 2476, 2021, 2009, 2001, 15966, 17579, 2084, 6548, 11651, 10975, 8447, 1998, 2062,

Since the length of the tokens is very large we simply truncate it to the 256 bits. We use Keras helper function to pad the sequences.

In [ ]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences
tokens_with_paading=pad_sequences(tokenized_reviews,maxlen=MAX_LENGTH,padding="post",truncating="post",value=0)
tokens_with_paading[700]
tokens_with_paading=np.array(tokens_with_paading)

In [ ]:
from sklearn.model_selection import train_test_split
train_inputs,test_inputs,train_labels,test_labels=train_test_split(tokens_with_paading,sentiment, test_size=0.15)

In [ ]:
train_inputs.shape

(42500, 256)

In [ ]:
def create_model(max_seq_len, bert_ckpt_file):

  with tf.io.gfile.GFile(bert_config_file, "r") as reader:
      stock_params = StockBertConfig.from_json_string(reader.read())
      bert_params  = stock_params.to_bert_model_layer_params()
      bert = BertModelLayer.from_params(bert_params, name="bert")
        
  input_ids = keras.layers.Input(shape=(max_seq_len, ), dtype='int32', name="input_ids")
  bert_output = bert(input_ids)

  print("bert shape", bert_output.shape)

  cls_out = keras.layers.Lambda(lambda seq: seq[:, 0, :])(bert_output)
  cls_out = keras.layers.Dropout(0.5)(cls_out)
  logits = keras.layers.Dense(units=768, activation="tanh")(cls_out)
  logits = keras.layers.Dropout(0.5)(logits)
  logits = keras.layers.Dense(units=1, activation="sigmoid")(logits)

  model = keras.Model(inputs=input_ids, outputs=logits)
  model.build(input_shape=(None, max_seq_len))

  load_stock_weights(bert, bert_ckpt_file)
        
  return model

In [ ]:
from bert import BertModelLayer
from bert.loader import StockBertConfig, load_stock_weights
model = create_model(MAX_LENGTH, bert_ckpt_file)

bert shape (None, 256, 768)
Done loading 196 BERT weights from: /content/model/uncased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7f53dc0b5da0> (prefix:bert). Count of weights not found in the checkpoint was: [0]. Count of weights with mismatched shape: [0]
Unused weights from checkpoint: 
	bert/embeddings/token_type_embeddings
	bert/pooler/dense/bias
	bert/pooler/dense/kernel
	cls/predictions/output_bias
	cls/predictions/transform/LayerNorm/beta
	cls/predictions/transform/LayerNorm/gamma
	cls/predictions/transform/dense/bias
	cls/predictions/transform/dense/kernel
	cls/seq_relationship/output_bias
	cls/seq_relationship/output_weights


In [ ]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_ids (InputLayer)       [(None, 256)]             0         
_________________________________________________________________
bert (BertModelLayer)        (None, 256, 768)          108890112 
_________________________________________________________________
lambda (Lambda)              (None, 768)               0         
_________________________________________________________________
dropout (Dropout)            (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 768)               590592    
_________________________________________________________________
dropout_1 (Dropout)          (None, 768)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                

In [ ]:
model.compile(
  optimizer=keras.optimizers.Adam(1e-5),
  loss=keras.losses.BinaryCrossentropy(from_logits=True),
  metrics=["accuracy"]
)

In [ ]:
import datetime
log_dir = "log/intent_detection/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%s")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir)

history = model.fit(
  x=train_inputs, 
  y=train_labels,
  validation_split=0.15,
  batch_size=16,
  shuffle=True,
  epochs=5,
  callbacks=[tensorboard_callback]
)


Epoch 1/5
2258/2258 [==============================] - 2695s 1s/step - loss: 0.5715 - accuracy: 0.8444 - val_loss: 0.5489 - val_accuracy: 0.9043
Epoch 2/5
2258/2258 [==============================] - 2703s 1s/step - loss: 0.5470 - accuracy: 0.9050 - val_loss: 0.5703 - val_accuracy: 0.8257
Epoch 3/5
2258/2258 [==============================] - 2696s 1s/step - loss: 0.5432 - accuracy: 0.9140 - val_loss: 0.5616 - val_accuracy: 0.8955
Epoch 4/5
2258/2258 [==============================] - 2693s 1s/step - loss: 0.5402 - accuracy: 0.9195 - val_loss: 0.5441 - val_accuracy: 0.9153
Epoch 5/5
2258/2258 [==============================] - 2676s 1s/step - loss: 0.5380 - accuracy: 0.9233 - val_loss: 0.5452 - val_accuracy: 0.9035


In [ ]:
model.save_weights("sentiment_bert_weights.h5")

In [ ]:
files.upload()

Saving sentiment_bert_weights.h5 to sentiment_bert_weights.h5


In [ ]:
model.load_weights("sentiment_bert_weights.h5")

OSError: ignored

In [ ]:
_, train_acc = model.evaluate(train_inputs, train_labels)
_, test_acc = model.evaluate(test_inputs, test_labels)

print("train acc", train_acc)
print("test acc", test_acc)

235/235 [==============================] - 147s 624ms/step - loss: 0.5448 - accuracy: 0.9035
train acc 0.9323999881744385
test acc 0.9034666419029236


In [ ]:
list_test=[]
raw_text_2="Hey my_name is prakash. I am not just doing a NLP intro."
preprocess_raw_text=preprocess_text(raw_text_2)
print(preprocess_raw_text)
list_test.append(preprocess_raw_text)
print(list_test)
tokenize_raw_text=tokenize_reviews(list_test)
print(tokenize_raw_text)

Hey my name is prakash I am not just doing a NLP intro 
['Hey my name is prakash I am not just doing a NLP intro ']
[[101, 4931, 2026, 2171, 2003, 22233, 1045, 2572, 2025, 2074, 2725, 1037, 17953, 2361, 17174, 102]]


In [ ]:
test_input_with_padding=pad_sequences(tokenize_raw_text,maxlen=MAX_LENGTH,padding="post",truncating="post",value=0)
test_input_with_padding=np.array(test_input_with_padding)
test_input_with_padding.shape

(1, 256)

In [ ]:
prediction=model.predict(test_input_with_padding)
print(prediction)

[[0.9999999]]


In [ ]:
train_labels.shape

(42500,)

In [ ]:
test_str="In the United Kingdom, periodicals like The Strand Magazine and Story-Teller contributed to the popularity of the short story. Hector Hugh Munro (1870–1916), also known by his pen name of Saki, wrote satirical short stories about Edwardian England. W. Somerset Maugham, who wrote over a hundred short stories, was one of the most popular authors of his time. P.G. Wodehouse published his first collection of comical stories about valet Jeeves in 1917. Many detective stories were written by G.K. Chesterton, Agatha Christie and Dorothy L. Sayers. Short stories by Virginia Woolf are Kew Gardens (1919) and Solid Objects, about a politician with mental problems. Graham Greene wrote his Twenty-One Stories between 1929 and 1954. A specialist in the short story was V.S. Pritchett, whose first collection appeared in 1932. Arthur C. Clarke published his first science fiction story, Travel by Wire in 1937. Evelyn Waugh, Muriel Spark and L.P. Hartley were other popular British storytellers whose career started in this period."
print(len(test_str))
tokens=tokenizer.tokenize(test_str)
print(tokens)
print(len(tokens))


1025
['in', 'the', 'united', 'kingdom', ',', 'periodicals', 'like', 'the', 'strand', 'magazine', 'and', 'story', '-', 'teller', 'contributed', 'to', 'the', 'popularity', 'of', 'the', 'short', 'story', '.', 'hector', 'hugh', 'munro', '(', '1870', '–', '1916', ')', ',', 'also', 'known', 'by', 'his', 'pen', 'name', 'of', 'sa', '##ki', ',', 'wrote', 'satirical', 'short', 'stories', 'about', 'edward', '##ian', 'england', '.', 'w', '.', 'somerset', 'ma', '##ugh', '##am', ',', 'who', 'wrote', 'over', 'a', 'hundred', 'short', 'stories', ',', 'was', 'one', 'of', 'the', 'most', 'popular', 'authors', 'of', 'his', 'time', '.', 'p', '.', 'g', '.', 'wo', '##deh', '##ouse', 'published', 'his', 'first', 'collection', 'of', 'comical', 'stories', 'about', 'valet', 'je', '##eves', 'in', '1917', '.', 'many', 'detective', 'stories', 'were', 'written', 'by', 'g', '.', 'k', '.', 'chester', '##ton', ',', 'agatha', 'christie', 'and', 'dorothy', 'l', '.', 'say', '##ers', '.', 'short', 'stories', 'by', 'virginia

In [ ]:
raw_text_1="One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fact that it goes where other shows wouldn't dare. Forget pretty pictures painted for mainstream audiences, forget charm, forget romance...OZ doesn't mess around. The first episode I ever saw struck me as so nasty it was surreal, I couldn't say I was ready for it, but as I watched more, I developed a taste for Oz, and got accustomed to the high levels of graphic violence. Not just violence, but injustice (crooked guards who'll be sold out for a nickel, inmates who'll kill on order and get away with it, well mannered, middle class inmates being turned into prison bitches due to their lack of street skills or prison experience) Watching Oz, you may become comfortable with what is uncomfortable viewing....thats if you can get in touch with your darker side."
raw_text_2="Hey my_name is prakash. I am not just doing a NLP intro."

In [ ]:
process_text_1=preprocess_text(raw_text_1)
process_text_2=preprocess_text(raw_text_2)

In [ ]:
test_tokens=[]
tokens_1=list(tokenizer.tokenize(process_text_1))
tokens_2=list(tokenizer.tokenize(process_text_2))
print(tokens_1)
print(tokens_2)
test_tokens.append(tokens_1)
test_tokens.append(tokens_2)
print(len(test_tokens))

['one', 'of', 'the', 'other', 'reviewers', 'has', 'mentioned', 'that', 'after', 'watching', 'just', 'oz', 'episode', 'you', 'll', 'be', 'hooked', 'they', 'are', 'right', 'as', 'this', 'is', 'exactly', 'what', 'happened', 'with', 'me', 'the', 'first', 'thing', 'that', 'struck', 'me', 'about', 'oz', 'was', 'its', 'brutality', 'and', 'un', '##fl', '##in', '##ching', 'scenes', 'of', 'violence', 'which', 'set', 'in', 'right', 'from', 'the', 'word', 'go', 'trust', 'me', 'this', 'is', 'not', 'a', 'show', 'for', 'the', 'faint', 'hearted', 'or', 'tim', '##id', 'this', 'show', 'pulls', 'no', 'punches', 'with', 'regards', 'to', 'drugs', 'sex', 'or', 'violence', 'its', 'is', 'hardcore', 'in', 'the', 'classic', 'use', 'of', 'the', 'word', 'it', 'is', 'called', 'oz', 'as', 'that', 'is', 'the', 'nickname', 'given', 'to', 'the', 'oswald', 'maximum', 'security', 'state', 'pen', '##ite', '##nta', '##ry', 'it', 'focuses', 'mainly', 'on', 'emerald', 'city', 'an', 'experimental', 'section', 'of', 'the', 'p

In [ ]:
print(len(tokens_1))
print(len(tokens_2))
print(len(test_tokens))


330
14
2


In [ ]:
for test_token in test_tokens:
  if len(test_token)>254:
    test_token=test_token[:254]

In [ ]:
len(test_tokens[0])

330